In [1]:
import pandas as pd 

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Input

In [2]:
def not_bunny(data): # Not bunny is normal LSTM model
    
    # Scaling the data
    scaler = MinMaxScaler(feature_range=(0,1))
    data = scaler.fit_transform(data)
    
    # Split the data into training and testing
    train_size = int(len(data) * 0.8)
    test_size = len(data) - train_size
    
    train, test = data[0:train_size, :], data[train_size:len(data), :]
    
    # Create the data structure
    pred_days = 20 # number of days that I wanna look back to predict the next day
    X_train, y_train = [], []
    
    for x in range(pred_days, len(train)):
        X_train.append(train[x-pred_days:x, 0])
        y_train.append(train[x, 0])
        
    X_train, y_train = np.array(X_train), np.array(y_train) # convert to numpy array so that we can use it to train the LSTM model
    
    # Reshape the data
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1)) # LSTM model expects 3D data in the form of (number of samples, number of time steps, number of features)
    
    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(units=10, return_sequences=True, input_shape=(X_train.shape[1], 1))) # 10 neurons in the first layer
    model.add(Dense(units=10)) # 10 neurons in the second layer
    model.add(Dropout(0.5)) # Dropout layer to prevent overfitting
    model.add(Dense(units=1)) # 1 neuron in the output layer
    
    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    # Train the model
    model.fit(X_train, y_train, epochs=10, batch_size=32)
    
    # Create the testing data
    X_test, y_test = [], []
    
    for x in range(pred_days, len(test)):
        X_test.append(test[x-pred_days:x, 0])
        y_test.append(test[x, 0])
        
    X_test, y_test = np.array(X_test), np.array(y_test) # convert to numpy array so that we can use it to test the LSTM model
    
    # Reshape the data
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1)) # LSTM model expects 3D data in the form of (number of samples, number of time steps, number of features)
    
    # Get the predicted stock price
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions) # unscale the data
    
    return predictions

In [3]:
data = pd.read_csv('stock_data.csv')
data

,Date,Ticker,Adj Close,Close,High,Low,Open,Volume
0,2014-02-03,AAPL,15.703114,17.911785,18.133215,17.832144,17.950357,401464000
1,2014-02-03,AMD,3.330000,3.330000,3.440000,3.330000,3.430000,22280200
2,2014-02-03,AMZN,17.307501,17.307501,18.042500,17.004999,17.948999,205616000
3,2014-02-03,CRM,58.470001,58.470001,61.200001,58.070000,61.099998,5450000
4,2014-02-03,GOOGL,28.364115,28.364115,29.572573,28.328579,29.509510,182581236
...,...,...,...,...,...,...,...,...
25165,2024-02-01,INTC,43.360001,43.360001,43.570000,42.490002,43.150002,38819900
25166,2024-02-01,META,394.779999,394.779999,400.500000,393.049988,393.940002,29727100
25167,2024-02-01,MSFT,403.779999,403.779999,408.000000,401.799988,401.829987,30657700
25168,2024-02-01,NVDA,630.270020,630.270020,631.909973,616.500000,621.000000,36914600


In [4]:
del data['Volume']
data

,Date,Ticker,Adj Close,Close,High,Low,Open
0,2014-02-03,AAPL,15.703114,17.911785,18.133215,17.832144,17.950357
1,2014-02-03,AMD,3.330000,3.330000,3.440000,3.330000,3.430000
2,2014-02-03,AMZN,17.307501,17.307501,18.042500,17.004999,17.948999
3,2014-02-03,CRM,58.470001,58.470001,61.200001,58.070000,61.099998
4,2014-02-03,GOOGL,28.364115,28.364115,29.572573,28.328579,29.509510
...,...,...,...,...,...,...,...
25165,2024-02-01,INTC,43.360001,43.360001,43.570000,42.490002,43.150002
25166,2024-02-01,META,394.779999,394.779999,400.500000,393.049988,393.940002
25167,2024-02-01,MSFT,403.779999,403.779999,408.000000,401.799988,401.829987
25168,2024-02-01,NVDA,630.270020,630.270020,631.909973,616.500000,621.000000


In [5]:
data = data.set_index(['Date', 'Ticker'])

In [6]:
data

Adj Close       Close        High         Low        Open
Date       Ticker                                                            
2014-02-03 AAPL     15.703114   17.911785   18.133215   17.832144   17.950357
           AMD       3.330000    3.330000    3.440000    3.330000    3.430000
           AMZN     17.307501   17.307501   18.042500   17.004999   17.948999
           CRM      58.470001   58.470001   61.200001   58.070000   61.099998
           GOOGL    28.364115   28.364115   29.572573   28.328579   29.509510
...                       ...         ...         ...         ...         ...
2024-02-01 INTC     43.360001   43.360001   43.570000   42.490002   43.150002
           META    394.779999  394.779999  400.500000  393.049988  393.940002
           MSFT    403.779999  403.779999  408.000000  401.799988  401.829987
           NVDA    630.270020  630.270020  631.909973  616.500000  621.000000
           TSLA    188.860001  188.860001  189.880005  184.279999  188.500000

[25170 rows x 5 columns]

In [7]:
data_aapl = data.xs('AAPL', level='Ticker')
data_aapl

,Adj Close,Close,High,Low,Open
Date,,,,,
2014-02-03,15.703114,17.911785,18.133215,17.832144,17.950357
2014-02-04,15.930429,18.171070,18.195000,17.955713,18.066071
2014-02-05,16.049398,18.306786,18.402857,18.080357,18.091429
2014-02-06,16.142948,18.303928,18.339287,18.136070,18.216429
2014-02-07,16.368795,18.559999,18.676071,18.477858,18.620714
...,...,...,...,...,...
2024-01-26,192.419998,192.419998,194.759995,191.940002,194.270004
2024-01-29,191.729996,191.729996,192.199997,189.580002,192.009995
2024-01-30,188.039993,188.039993,191.800003,187.470001,190.940002


In [8]:
aapl_close = data_aapl['Adj Close']
aapl_close = aapl_close.values
aapl_close

array([ 15.70311356,  15.9304285 ,  16.04939842, ..., 188.03999329,
       184.3999939 , 186.86000061])

In [9]:
pred = not_bunny(data_aapl)



Epoch 1/10

63/63 [==============================] - 2s 4ms/step - loss: 0.0613
Epoch 2/10
63/63 [==============================] - 0s 4ms/step - loss: 0.0553
Epoch 3/10
63/63 [==============================] - 0s 5ms/step - loss: 0.0542
Epoch 4/10
63/63 [==============================] - 0s 4ms/step - loss: 0.0535
Epoch 5/10
63/63 [==============================] - 0s 4ms/step - loss: 0.0531
Epoch 6/10
63/63 [==============================] - 0s 4ms/step - loss: 0.0527
Epoch 7/10
63/63 [==============================] - 0s 4ms/step - loss: 0.0525
Epoch 8/10
63/63 [==============================] - 0s 4ms/step - loss: 0.0524
Epoch 9/10
63/63 [==============================] - 0s 4ms/step - loss: 0.0522
Epoch 10/10
16/16 [==============================] - 0s 2ms/step


ValueError: Found array with dim 3. None expected <= 2.